# Notebook dev

NB : all variables & filters using the word CDS are corresponding to the TER concept in the paper

In [1]:
# General imports
import os
import sys
import pandas as pd
import scipy
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np
from tqdm import tqdm
sys.path.append('../')
# Other imports

tqdm.pandas()
import yaml
import json


## YAML FILES CONFIG
yaml = yaml.load(open("config/config_files.yaml"), Loader=yaml.FullLoader)
base_dir = yaml['base_directory']
sys.exit('EXIT : Need to specify the base_directory in config file : "conf_files.yaml"') if base_dir == 'TO_CHANGE' else None
    

# Define Miso & Siso

In [3]:
genes_raw = pd.read_csv(base_dir + yaml['External']['biomart_ensembl'], compression='gzip', sep='\t')
genes_raw['Chromosome/scaffold name'] = genes_raw['Chromosome/scaffold name'].astype(str)
genes_raw = genes_raw.loc[~genes_raw['Chromosome/scaffold name'].str.contains('CHR|\.')].copy()
genes = genes_raw[['Gene stable ID', 'Gene name', 'Chromosome/scaffold name', 'Gene start (bp)', 'Gene end (bp)']].drop_duplicates().sort_values(by='Gene name')
genes['Gene_length'] = genes['Gene end (bp)'] - genes['Gene start (bp)']
print('Genes : {}'.format(genes.shape[0]), )

mrna = genes_raw[['Gene stable ID', 'Transcript stable ID', 'Protein stable ID', 'Transcript start (bp)', 'Transcript end (bp)', 'Transcription start site (TSS)', 'Transcript length (including UTRs and CDS)', 'Transcript support level (TSL)', 'GENCODE basic annotation', 'APPRIS annotation']]
print('mRNA : {}'.format(mrna.shape[0]), )

miso_siso_raw = mrna[['Gene stable ID', 'Transcript stable ID']].groupby('Gene stable ID')['Transcript stable ID'].nunique().reset_index()
miso_siso_raw.loc[miso_siso_raw['Transcript stable ID'] > 1, 'Raw_Miso_siso'] = 'Miso'
miso_siso_raw.loc[miso_siso_raw['Transcript stable ID'] == 1, 'Raw_Miso_siso'] = 'Siso'
miso_siso_raw = miso_siso_raw.rename({'Gene stable ID': 'GeneID', 'Transcript stable ID' : 'transcript_count_raw'}, axis=1)
# print('MISOG SISOG : {}'.format(mrna.shape[0]), mrna.columns)
print('Genes : ', miso_siso_raw.groupby('Raw_Miso_siso')['GeneID'].nunique())
print('mRNA : ',miso_siso_raw.groupby('Raw_Miso_siso')['transcript_count_raw'].sum())

/home/weber/.conda/envs/ExoCarto/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3166: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Genes : 19664
mRNA : 61295
Genes :  Raw_Miso_siso
Miso    13552
Siso     6112
Name: GeneID, dtype: int64
mRNA :  Raw_Miso_siso
Miso    55146
Siso     6112
Name: transcript_count_raw, dtype: int64


In [71]:
genes_raw

,Gene stable ID,Transcript stable ID,Protein stable ID,Gene description,Chromosome/scaffold name,Gene start (bp),Gene end (bp),Strand,Transcript start (bp),Transcript end (bp),...,Source of gene name,Transcript name,Source of transcript name,Gene % GC content,Gene type,Transcript type,Source (gene),Source (transcript),CCDS ID,UniProtKB isoform ID
0,ENSG00000198888,ENST00000361390,ENSP00000354687,mitochondrially encoded NADH:ubiquinone oxidor...,MT,3307,4262,1,3307,4262,...,HGNC Symbol,MT-ND1-201,Transcript name,47.70,protein_coding,protein_coding,insdc,insdc,NaN,NaN
1,ENSG00000198763,ENST00000361453,ENSP00000355046,mitochondrially encoded NADH:ubiquinone oxidor...,MT,4470,5511,1,4470,5511,...,HGNC Symbol,MT-ND2-201,Transcript name,42.99,protein_coding,protein_coding,insdc,insdc,NaN,NaN
2,ENSG00000198804,ENST00000361624,ENSP00000354499,mitochondrially encoded cytochrome c oxidase I...,MT,5904,7445,1,5904,7445,...,HGNC Symbol,MT-CO1-201,Transcript name,46.24,protein_coding,protein_coding,insdc,insdc,NaN,NaN
3,ENSG00000198712,ENST00000361739,ENSP00000354876,mitochondrially encoded cytochrome c oxidase I...,MT,7586,8269,1,7586,8269,...,HGNC Symbol,MT-CO2-201,Transcript name,46.20,protein_coding,protein_coding,insdc,insdc,NaN,NaN
4,ENSG00000228253,ENST00000361851,ENSP00000355265,mitochondrially encoded ATP synthase membrane ...,MT,8366,8572,1,8366,8572,...,HGNC Symbol,MT-ATP8-201,Transcript name,39.61,protein_coding,protein_coding,insdc,insdc,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68279,ENSG00000116754,ENST00000395136,ENSP00000378568,serine and arginine rich splicing factor 11 [S...,1,70205682,70253052,1,70221396,70250805,...,HGNC Symbol,SRSF11-204,Transcript name,37.01,protein_coding,protein_coding,ensembl_havana,havana,NaN,NaN
68280,ENSG00000283324,ENST00000636087,ENSP00000490418,cortexin domain containing 2 [Source:HGNC Symb...,1,150887136,150913292,1,150887136,150913292,...,HGNC Symbol,CTXND2-201,Transcript name,42.36,protein_coding,protein_coding,havana,havana,NaN,NaN
68281,ENSG00000077585,ENST00000366592,ENSP00000355551,G protein-coupled receptor 137B [Source:HGNC S...,1,236142505,236221865,1,236142539,236208907,...,HGNC Symbol,GPR137B-202,Transcript name,43.63,protein_coding,protein_coding,ensembl_havana,ensembl_havana,CCDS1609,NaN
68282,ENSG00000086619,ENST00000354619,ENSP00000346635,endoplasmic reticulum oxidoreductase 1 beta [S...,1,236214681,236282019,-1,236215101,236281958,...,HGNC Symbol,ERO1B-202,Transcript name,37.46,protein_coding,protein_coding,ensembl_havana,ensembl_havana,CCDS31064,Q86YB8-1


In [4]:
gtex_checked = pd.read_csv('/gstock/EXOTIC/data/EXPRESSION/GTEx_V8_transcript_checking.gct.gz', compression='gzip', sep='\t')
gtex_checked['transcript_id'] = gtex_checked['transcript_id'].apply(lambda r : r.split('.')[0])

mrna = mrna.rename({'Gene stable ID': 'GeneID', 'Transcript stable ID' : 'transcript_id'}, axis=1)

mrna = mrna.loc[mrna['transcript_id'].isin(gtex_checked.loc[gtex_checked['check'] == True]['transcript_id'].values.tolist())]


miso_siso = mrna[['GeneID', 'transcript_id']].groupby('GeneID')['transcript_id'].nunique().reset_index()
miso_siso.loc[miso_siso['transcript_id'] > 1, 'Miso_siso'] = 'Miso'
miso_siso.loc[miso_siso['transcript_id'] == 1, 'Miso_siso'] = 'Siso'
miso_siso = miso_siso.rename({'transcript_id' : 'transcript_count'}, axis=1)
miso_siso.groupby('Miso_siso')['transcript_count'].sum()
print('GTEx Genes : ', miso_siso.groupby('Miso_siso')['GeneID'].nunique().sum())
print('GTEx Genes : ', miso_siso.groupby('Miso_siso')['GeneID'].nunique())
print('GTEx mRNA : ',miso_siso.groupby('Miso_siso')['transcript_count'].sum().sum())
print('GTEx mRNA : ',miso_siso.groupby('Miso_siso')['transcript_count'].sum())

GTEx Genes :  16172
GTEx Genes :  Miso_siso
Miso    9826
Siso    6346
Name: GeneID, dtype: int64
GTEx mRNA :  37433
GTEx mRNA :  Miso_siso
Miso    31087
Siso     6346
Name: transcript_count, dtype: int64


In [5]:
merge_raw_gtex = pd.merge(miso_siso, miso_siso_raw, on=['GeneID'])
merge_raw_gtex.loc[merge_raw_gtex['Miso_siso'] == merge_raw_gtex['Raw_Miso_siso'], 'Same'] = True
merge_raw_gtex.loc[merge_raw_gtex['Miso_siso'] != merge_raw_gtex['Raw_Miso_siso'], 'Same'] = False
genes_to_keep = merge_raw_gtex.loc[merge_raw_gtex['Same'] == True]['GeneID'].values.tolist()
genes_to_not_keep = merge_raw_gtex.loc[merge_raw_gtex['Same'] == False]['GeneID'].values.tolist()
print(len(genes_to_not_keep))
mrna = mrna.loc[mrna['GeneID'].isin(genes_to_keep)]
mrna_miso_siso = pd.merge(mrna, miso_siso, on=['GeneID'])

2281


In [92]:
mrna_miso_siso[['GeneID', 'transcript_count', 'Miso_siso']].drop_duplicates().groupby('Miso_siso')['transcript_count'].sum()

Miso_siso
Miso    31087
Siso     4065
Name: transcript_count, dtype: int64

In [117]:
len(genes_to_not_keep)

2281

In [87]:
merge_raw_gtex.Raw_Miso_siso.value_counts()

Miso    12107
Siso     4065
Name: Raw_Miso_siso, dtype: int64

In [82]:
merge_raw_gtex.Miso_siso.value_counts()

Miso    9826
Siso    6346
Name: Miso_siso, dtype: int64

In [86]:
merge_raw_gtex.loc[merge_raw_gtex['Same'] == True]['Miso_siso'].value_counts()

Miso    9826
Siso    4065
Name: Miso_siso, dtype: int64

In [96]:
merge_raw_gtex[['GeneID', 'transcript_count', 'Miso_siso']].drop_duplicates()['transcript_count'].sum()

35152

In [93]:
merge_raw_gtex.loc[merge_raw_gtex['Same'] == True][['GeneID', 'transcript_count', 'Miso_siso']].drop_duplicates().groupby('Miso_siso')['transcript_count'].sum()

Miso_siso
Miso    31087
Siso     4065
Name: transcript_count, dtype: int64

In [97]:
mrna_miso_siso.groupby('Miso_siso')['transcript_count'].describe()

,count,mean,std,min,25%,50%,75%,max
Miso_siso,,,,,,,,
Miso,31111.0,3.975025,2.305804,2.0,2.0,3.0,5.0,21.0
Siso,4065.0,1.000000,0.000000,1.0,1.0,1.0,1.0,1.0


In [3]:
# exons = pd.read_csv('/gstock/GeneIso/data/External/ENSEMBL/mart_export_exon_38.txt.gz', compression='gzip', sep='\t')
exons = pd.read_csv(base_dir + yaml['External']['biomart_ensembl_exons'], compression='gzip', sep='\t')

exons = exons.rename({'Gene stable ID': 'GeneID', 'Transcript stable ID' : 'transcript_id'}, axis=1)

merge_exon_miso_siso = pd.merge(mrna_miso_siso, exons, on=['GeneID', 'transcript_id'])


tmp = merge_exon_miso_siso[['GeneID', 'Chromosome/scaffold name', 'Exon region start (bp)', 'Exon region end (bp)', 'Miso_siso']].drop_duplicates().groupby(['Miso_siso', 'GeneID'])['Exon region start (bp)'].count().reset_index()
sisog_single_ter = tmp.loc[tmp['Exon region start (bp)'] == 1]['GeneID'].values.tolist()
print(len(sisog_single_ter))

In [4]:
exons.columns.tolist()

['Gene stable ID',
 'Transcript stable ID',
 'Exon region start (bp)',
 'Exon region end (bp)',
 "5' UTR start",
 "5' UTR end",
 "3' UTR start",
 "3' UTR end",
 'CDS Length',
 'Strand',
 'Transcription start site (TSS)',
 'Transcript length (including UTRs and CDS)',
 'Transcript start (bp)',
 'Transcript end (bp)',
 'Gene start (bp)',
 'Gene end (bp)',
 'Chromosome/scaffold name',
 'CDS start',
 'CDS end',
 'Constitutive exon',
 'Exon rank in transcript',
 'Start phase',
 'End phase',
 'cDNA coding start',
 'cDNA coding end',
 'Genomic coding start',
 'Genomic coding end',
 'Exon stable ID']

In [7]:
miso_siso_to_keep = miso_siso.loc[~miso_siso['GeneID'].isin(sisog_single_ter + genes_to_not_keep)]
print('Genes : ', miso_siso_to_keep.shape[0])

Genes :  13501


In [8]:
genes_miso_siso = pd.merge(miso_siso_to_keep, genes.rename({'Gene stable ID': 'GeneID'}, axis=1), on ='GeneID')
genes_miso_siso.groupby('Miso_siso')['Gene_length'].describe()

,count,mean,std,min,25%,50%,75%,max
Miso_siso,,,,,,,,
Miso,9826.0,82043.624364,143420.177937,704.0,15084.5,36573.5,89611.75,2473538.0
Siso,3675.0,51655.191837,90222.618571,489.0,9536.5,23037.0,56162.00,1899593.0


In [10]:
mrna_miso_siso = pd.merge(miso_siso_to_keep, mrna.rename({'Gene stable ID': 'GeneID'}, axis=1), on ='GeneID')
mrna_miso_siso.groupby('Miso_siso')['Transcript length (including UTRs and CDS)'].describe()

,count,mean,std,min,25%,50%,75%,max
Miso_siso,,,,,,,,
Miso,31111.0,3069.642249,2360.576207,267.0,1537.0,2424.0,3925.0,34626.0
Siso,3675.0,3728.145034,2771.967276,303.0,1805.0,2979.0,4865.0,30609.0


In [13]:
genes_miso_siso

,GeneID,transcript_count,Miso_siso,Gene name,Chromosome/scaffold name,Gene start (bp),Gene end (bp),Gene_length
0,ENSG00000000003,3,Miso,TSPAN6,X,100627108,100639991,12883
1,ENSG00000000005,1,Siso,TNMD,X,100584936,100599885,14949
2,ENSG00000000419,3,Miso,DPM1,20,50934867,50959140,24273
3,ENSG00000000457,3,Miso,SCYL3,1,169849631,169894267,44636
4,ENSG00000000460,4,Miso,C1orf112,1,169662007,169854080,192073
...,...,...,...,...,...,...,...,...
13496,ENSG00000286143,1,Siso,NaN,2,219491867,219498244,6377
13497,ENSG00000286185,1,Siso,NaN,1,149390623,149556361,165738
13498,ENSG00000286190,1,Siso,NaN,17,5499427,5501006,1579
13499,ENSG00000287542,1,Siso,NaN,4,88523810,88708450,184640


In [18]:
genes_miso_siso.to_parquet('/gstock/GeneIso/V2/Genes.parquet')
mrna_miso_siso.to_parquet('/gstock/GeneIso/V2/mRNA.parquet')


In [49]:
mrna_miso_siso.loc[mrna_miso_siso['Miso_siso'] == 'Miso'].groupby('GeneID')['transcript_id'].nunique().describe()

count    9826.000000
mean        3.163749
std         1.602909
min         2.000000
25%         2.000000
50%         3.000000
75%         4.000000
max        21.000000
Name: transcript_id, dtype: float64

In [226]:
miso_siso.loc[miso_siso['Miso_siso'] == 'Miso']['transcript_count'].describe().round()

count   9,149.00
mean        3.00
std         1.00
min         2.00
25%         2.00
50%         3.00
75%         4.00
max        21.00
Name: transcript_count, dtype: float64

In [14]:
exon_miso_siso_to_keep = pd.merge(exons, miso_siso_to_keep, on='GeneID')
exon_miso_siso_to_keep['Exon_length'] = exon_miso_siso_to_keep['Exon region end (bp)'] - exon_miso_siso_to_keep['Exon region start (bp)']
# exon_miso_siso_to_keep.drop_duplicates(subset=['GeneID', 'Chromosome/scaffold name', 'Exon region start (bp)', 'Exon region end (bp)']).groupby(['Miso_siso'])['Exon region start (bp)'].count()
# exon_miso_siso_to_keep.drop_duplicates(subset=['GeneID', 'Chromosome/scaffold name', 'Exon region start (bp)', 'Exon region end (bp)']).groupby(['Miso_siso', 'GeneID'])['Exon region start (bp)'].count().groupby('Miso_siso').describe()
# exon_miso_siso_to_keep

In [60]:
exon_miso_siso_to_keep.drop_duplicates(subset=['GeneID', 'Chromosome/scaffold name', 'Exon region start (bp)', 'Exon region end (bp)']).groupby('Miso_siso')['Exon_length'].describe()

,count,mean,std,min,25%,50%,75%,max
Miso_siso,,,,,,,,
Miso,183882.0,409.412607,921.972520,0.0,95.0,145.0,269.0,33289.0
Siso,36173.0,377.761314,961.647594,2.0,97.0,137.0,203.0,27302.0


In [63]:
exon_miso_siso_to_keep.columns

Index(['GeneID', 'transcript_id', 'Exon region start (bp)',
       'Exon region end (bp)', '5' UTR start', '5' UTR end', '3' UTR start',
       '3' UTR end', 'CDS Length', 'Strand', 'Transcription start site (TSS)',
       'Transcript length (including UTRs and CDS)', 'Transcript start (bp)',
       'Transcript end (bp)', 'Gene start (bp)', 'Gene end (bp)',
       'Chromosome/scaffold name', 'CDS start', 'CDS end', 'Constitutive exon',
       'Exon rank in transcript', 'Start phase', 'End phase',
       'cDNA coding start', 'cDNA coding end', 'Genomic coding start',
       'Genomic coding end', 'Exon stable ID', 'transcript_count', 'Miso_siso',
       'Exon_length'],
      dtype='object')

In [75]:
exon_tmp_dev

,GeneID,transcript_id,Strand,Chromosome/scaffold name,Exon region start (bp),Exon region end (bp),5' UTR start,5' UTR end,3' UTR start,3' UTR end,CDS start,CDS end,Miso_siso
0,ENSG00000198692,ENST00000361365,1,Y,20575776,20575887,20575776.0,20575871.0,NaN,NaN,1.0,16.0,Miso
1,ENSG00000198692,ENST00000361365,1,Y,20579608,20579691,NaN,NaN,NaN,NaN,17.0,100.0,Miso
2,ENSG00000198692,ENST00000361365,1,Y,20582590,20582693,NaN,NaN,NaN,NaN,101.0,204.0,Miso
3,ENSG00000198692,ENST00000361365,1,Y,20584474,20584524,NaN,NaN,NaN,NaN,205.0,255.0,Miso
4,ENSG00000198692,ENST00000361365,1,Y,20588024,20588105,NaN,NaN,NaN,NaN,256.0,337.0,Miso
...,...,...,...,...,...,...,...,...,...,...,...,...,...
536344,ENSG00000077585,ENST00000366592,1,1,236178414,236178636,NaN,NaN,NaN,NaN,465.0,687.0,Siso
536345,ENSG00000077585,ENST00000366592,1,1,236179879,236180028,NaN,NaN,NaN,NaN,688.0,837.0,Siso
536346,ENSG00000077585,ENST00000366592,1,1,236183778,236183906,NaN,NaN,NaN,NaN,838.0,966.0,Siso
536347,ENSG00000077585,ENST00000366592,1,1,236205126,236205250,NaN,NaN,NaN,NaN,967.0,1091.0,Siso


In [87]:
exon_tmp_dev.sort_values(by=['Chromosome/scaffold name', 'Exon region start (bp)', 'Exon region end (bp)', 'GeneID', 'transcript_id']).head(100).dropna(subset=["5' UTR end"]).groupby('transcript_id').apply(lambda r: r["5' UTR end"].values.tolist())

transcript_id
ENST00000304952               [1000097.0]
ENST00000327044                [959256.0]
ENST00000338591                [960693.0]
ENST00000341290      [981173.0, 982093.0]
ENST00000379370               [1020172.0]
ENST00000379407                [966531.0]
ENST00000379409                [966531.0]
ENST00000379410                [966531.0]
ENST00000428771               [1000172.0]
ENST00000484667                [999981.0]
ENST00000620552    [1020373.0, 1022413.0]
dtype: object

In [15]:
from sqlalchemy import create_engine
engine = create_engine("sqlite:////gstock/GeneIso/V2/GeneIso.db")

In [34]:

def exon_ord(df):
    if df.Strand.values[0] == 1:
        df = df.sort_values(by=['Exon region start (bp)', 'Exon region end (bp)'])

    elif df.Strand.values[0] == -1:
        df = df.sort_values(by=['Exon region start (bp)', 'Exon region end (bp)'], ascending=False)
    
    df['Ordinal_nb'] = list(range(1,df.shape[0]+1))
    df['Length'] = df['Exon region end (bp)'] - df['Exon region start (bp)']
    df['Ordinal_nb_inverted'] = np.array(list(range(1,df.shape[0] + 1))[::-1]) * -1

    return df[['Miso_siso', 'GeneID', 'transcript_id', 'Strand', 'Chromosome/scaffold name', 'Exon region start (bp)', 'Exon region end (bp)', 'Ordinal_nb', 'Ordinal_nb_inverted', 'Length']]
    

exon_tmp_dev = exon_miso_siso_to_keep[['GeneID', 'transcript_id', 'Strand', 'Chromosome/scaffold name', 'Exon region start (bp)', 'Exon region end (bp)', "5' UTR start", "5' UTR end", "3' UTR start", "3' UTR end", 'CDS start', 'CDS end', 'Miso_siso']]
exon_tmp_dev = exon_tmp_dev.sort_values(by=['Chromosome/scaffold name', 'Exon region start (bp)', 'Exon region end (bp)', 'GeneID', 'transcript_id'])

# exon_tmp_dev = exon_tmp_dev.loc[exon_tmp_dev['GeneID'].isin(exon_tmp_dev['GeneID'].unique().tolist()[:100])]

exon_tmp_dev = exon_tmp_dev.groupby(['Miso_siso', 'GeneID', 'transcript_id']).progress_apply(exon_ord).reset_index(drop=True)



exon_tmp_dev['Chromosome/scaffold name'] = exon_tmp_dev['Chromosome/scaffold name'].astype(str)

# exon_tmp_dev.to_sql('Exons', engine, if_exists='replace')

exon_tmp_dev.to_parquet('/gstock/GeneIso/V2/Exons.parquet')

exon_tmp_dev

100%|██████████| 47544/47544 [13:18<00:00, 59.53it/s] 


,Miso_siso,GeneID,transcript_id,Strand,Chromosome/scaffold name,Exon region start (bp),Exon region end (bp),Ordinal_nb,Ordinal_nb_inverted,Length
0,Miso,ENSG00000000003,ENST00000373020,-1,X,100636608,100636806,1,-8,198
1,Miso,ENSG00000000003,ENST00000373020,-1,X,100635558,100635746,2,-7,188
2,Miso,ENSG00000000003,ENST00000373020,-1,X,100635178,100635252,3,-6,74
3,Miso,ENSG00000000003,ENST00000373020,-1,X,100633931,100634029,4,-5,98
4,Miso,ENSG00000000003,ENST00000373020,-1,X,100633405,100633539,5,-4,134
...,...,...,...,...,...,...,...,...,...,...
536344,Siso,ENSG00000287542,ENST00000512194,1,4,88706752,88708450,26,-1,1698
536345,Siso,ENSG00000288920,ENST00000484897,1,19,23763004,23763134,1,-4,130
536346,Siso,ENSG00000288920,ENST00000484897,1,19,23807826,23807939,2,-3,113
536347,Siso,ENSG00000288920,ENST00000484897,1,19,23808719,23808814,3,-2,95


In [3]:
exon_tmp_dev = pd.read_parquet('/gstock/GeneIso/V2/Exons.parquet')

In [33]:
def retrieve_cds_coord(df):
    if df.Strand.values[0] == 1:
        df = df.sort_values(by=['Exon region start (bp)', 'Exon region end (bp)'])

        df_cds = df.dropna(subset=['CDS start', 'CDS end'])
        df_cds['Ordinal_nb'] = list(range(1,df_cds.shape[0]+1))
        df_cds['Ordinal_nb_inverted'] = np.array(list(range(1,df_cds.shape[0] + 1))[::-1]) * -1

        df_cds_start = df_cds.loc[df_cds['Ordinal_nb'] == 1]
        df_cds_start['CDS_start_abs'] = df_cds_start["5' UTR end"] + 1
        df_cds_start['CDS_end_abs'] = df_cds_start["Exon region end (bp)"]

        df_cds_core = df_cds.loc[~df_cds['Ordinal_nb'].isin([1, df_cds.shape[0]])]
        df_cds_core['CDS_start_abs'] = df_cds_core['Exon region start (bp)'] 
        df_cds_core['CDS_end_abs'] = df_cds_core['Exon region end (bp)'] 

        df_cds_end = df_cds.loc[df_cds['Ordinal_nb'] == df_cds.shape[0]]
        df_cds_end['CDS_start_abs'] = df_cds_end['Exon region start (bp)'] 
        df_cds_end['CDS_end_abs'] = df_cds_end["3' UTR start"] - 1

        df_cds_final = pd.concat([df_cds_start, df_cds_core, df_cds_end])

    elif df.Strand.values[0] == -1:
        df = df.sort_values(by=['Exon region start (bp)', 'Exon region end (bp)'], ascending=False)

        df_cds = df.dropna(subset=['CDS start', 'CDS end'])
        df_cds['Ordinal_nb'] = list(range(1,df_cds.shape[0]+1))
        df_cds['Ordinal_nb_inverted'] = np.array(list(range(1,df_cds.shape[0] + 1))[::-1]) * -1

        
        df_cds_start = df_cds.loc[df_cds['Ordinal_nb'] == 1]
        df_cds_start['CDS_start_abs'] = df_cds_start["Exon region start (bp)"]
        df_cds_start['CDS_end_abs'] = df_cds_start["5' UTR start"] - 1

        df_cds_core = df_cds.loc[~df_cds['Ordinal_nb'].isin([1, df_cds.shape[0]])]
        df_cds_core['CDS_start_abs'] = df_cds_core['Exon region start (bp)'] 
        df_cds_core['CDS_end_abs'] = df_cds_core['Exon region end (bp)'] 

        df_cds_end = df_cds.loc[df_cds['Ordinal_nb'] == df_cds.shape[0]]
        df_cds_end['CDS_start_abs'] = df_cds_end["3' UTR end"] + 1
        df_cds_end['CDS_end_abs'] = df_cds_end['Exon region end (bp)']

        df_cds_final = pd.concat([df_cds_start, df_cds_core, df_cds_end])

    df_cds_final['Length'] = df_cds_final['CDS end'] - df_cds_final['CDS start']
    return df_cds_final


exon_tmp_dev = exon_miso_siso_to_keep[['GeneID', 'transcript_id', 'Strand', 'Chromosome/scaffold name', 'Exon region start (bp)', 'Exon region end (bp)', "5' UTR start", "5' UTR end", "3' UTR start", "3' UTR end", 'CDS start', 'CDS end', 'Miso_siso']]
exon_tmp_dev = exon_tmp_dev.sort_values(by=['Chromosome/scaffold name', 'Exon region start (bp)', 'Exon region end (bp)', 'GeneID', 'transcript_id'])
exon_tmp_dev['Chromosome/scaffold name'] = exon_tmp_dev['Chromosome/scaffold name'].astype(str)

# exon_tmp_dev = exon_tmp_dev.loc[exon_tmp_dev['GeneID'].isin(exon_tmp_dev['GeneID'].unique().tolist()[:100])]

cds = exon_tmp_dev.groupby(['Miso_siso', 'GeneID', 'transcript_id']).progress_apply(retrieve_cds_coord).reset_index(drop=True)

# cds.to_sql('CDS', engine, if_exists='replace')
cds.to_parquet('/gstock/GeneIso/V2/CDS.parquet')

cds


100%|██████████| 47544/47544 [26:10<00:00, 30.27it/s]


,GeneID,transcript_id,Strand,Chromosome/scaffold name,Exon region start (bp),Exon region end (bp),5' UTR start,5' UTR end,3' UTR start,3' UTR end,CDS start,CDS end,Miso_siso,Ordinal_nb,Ordinal_nb_inverted,CDS_start_abs,CDS_end_abs,Length
0,ENSG00000000003,ENST00000373020,-1,X,100636608,100636806,100636695.0,100636806.0,NaN,NaN,1.0,87.0,Miso,1,-7,100636608.0,100636694.0,86.0
1,ENSG00000000003,ENST00000373020,-1,X,100635558,100635746,NaN,NaN,NaN,NaN,88.0,276.0,Miso,2,-6,100635558.0,100635746.0,188.0
2,ENSG00000000003,ENST00000373020,-1,X,100635178,100635252,NaN,NaN,NaN,NaN,277.0,351.0,Miso,3,-5,100635178.0,100635252.0,74.0
3,ENSG00000000003,ENST00000373020,-1,X,100633931,100634029,NaN,NaN,NaN,NaN,352.0,450.0,Miso,4,-4,100633931.0,100634029.0,98.0
4,ENSG00000000003,ENST00000373020,-1,X,100633405,100633539,NaN,NaN,NaN,NaN,451.0,585.0,Miso,5,-3,100633405.0,100633539.0,134.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
497254,ENSG00000287542,ENST00000512194,1,4,88704098,88704281,NaN,NaN,NaN,NaN,2634.0,2817.0,Siso,21,-3,88704098.0,88704281.0,183.0
497255,ENSG00000287542,ENST00000512194,1,4,88704508,88704610,NaN,NaN,NaN,NaN,2818.0,2920.0,Siso,22,-2,88704508.0,88704610.0,102.0
497256,ENSG00000287542,ENST00000512194,1,4,88706752,88708450,NaN,NaN,88706961.0,88708450.0,2921.0,3129.0,Siso,23,-1,88706752.0,88706960.0,208.0
497257,ENSG00000288920,ENST00000484897,1,19,23827070,23828868,23827070.0,23827161.0,23828050.0,23828868.0,1.0,888.0,Siso,1,-1,23827162.0,23828868.0,887.0


In [32]:
def retrieve_5_utr(df):
    if df.Strand.values[0] == 1:
        df = df.sort_values(by=['Exon region start (bp)', 'Exon region end (bp)'])

    elif df.Strand.values[0] == -1:
        df = df.sort_values(by=['Exon region start (bp)', 'Exon region end (bp)'], ascending=False)

    df_5_utr = df.dropna(subset=["5' UTR start", "5' UTR end"])
    df_5_utr['Ordinal_nb'] = list(range(1,df_5_utr.shape[0]+1))
    df_5_utr['Ordinal_nb_inverted'] = np.array(list(range(1,df_5_utr.shape[0] + 1))[::-1]) * -1
    
    df_5_utr['Length'] = df_5_utr["5' UTR end"] - df_5_utr["5' UTR start"]
    return df_5_utr

def retrieve_3_utr(df):
    if df.Strand.values[0] == 1:
        df = df.sort_values(by=['Exon region start (bp)', 'Exon region end (bp)'])

    elif df.Strand.values[0] == -1:
        df = df.sort_values(by=['Exon region start (bp)', 'Exon region end (bp)'], ascending=False)

    df_3_utr = df.dropna(subset=["3' UTR start", "3' UTR end"])
    df_3_utr['Ordinal_nb'] = list(range(1,df_3_utr.shape[0]+1))
    df_3_utr['Ordinal_nb_inverted'] = np.array(list(range(1,df_3_utr.shape[0] + 1))[::-1]) * -1
    
    df_3_utr['Length'] = df_3_utr["3' UTR end"] - df_3_utr["3' UTR start"]

    return df_3_utr

exon_tmp_dev = exon_miso_siso_to_keep[['GeneID', 'transcript_id', 'Strand', 'Chromosome/scaffold name', 'Exon region start (bp)', 'Exon region end (bp)', "5' UTR start", "5' UTR end", "3' UTR start", "3' UTR end", 'CDS start', 'CDS end', 'Miso_siso']]
exon_tmp_dev = exon_tmp_dev.sort_values(by=['Chromosome/scaffold name', 'Exon region start (bp)', 'Exon region end (bp)', 'GeneID', 'transcript_id'])

# exon_tmp_dev = exon_tmp_dev.loc[exon_tmp_dev['GeneID'].isin(exon_tmp_dev['GeneID'].unique().tolist()[:100])]

df_5_utr = exon_tmp_dev.groupby(['Miso_siso', 'GeneID', 'transcript_id']).progress_apply(retrieve_5_utr).reset_index(drop=True)
df_3_utr = exon_tmp_dev.groupby(['Miso_siso', 'GeneID', 'transcript_id']).progress_apply(retrieve_3_utr).reset_index(drop=True)


df_5_utr['Chromosome/scaffold name'] = df_5_utr['Chromosome/scaffold name'].astype(str)
df_3_utr['Chromosome/scaffold name'] = df_3_utr['Chromosome/scaffold name'].astype(str)


# df_5_utr.to_sql('5_UTR', engine, if_exists='replace')
df_5_utr.to_parquet('/gstock/GeneIso/V2/5_UTR.parquet')

# df_3_utr.to_sql('3_UTR', engine, if_exists='replace')
df_3_utr.to_parquet('/gstock/GeneIso/V2/3_UTR.parquet')

df_5_utr

100%|██████████| 47544/47544 [14:44<00:00, 53.76it/s] 


,GeneID,transcript_id,Strand,Chromosome/scaffold name,Exon region start (bp),Exon region end (bp),5' UTR start,5' UTR end,3' UTR start,3' UTR end,CDS start,CDS end,Miso_siso,Ordinal_nb,Ordinal_nb_inverted,Length
0,ENSG00000000003,ENST00000373020,-1,X,100636608,100636806,100636695.0,100636806.0,NaN,NaN,1.0,87.0,Miso,1.0,-1.0,111.0
1,ENSG00000000003,ENST00000612152,-1,X,100636793,100637104,100636793.0,100637104.0,NaN,NaN,NaN,NaN,Miso,1.0,-2.0,311.0
2,ENSG00000000003,ENST00000612152,-1,X,100635558,100635746,100635570.0,100635746.0,NaN,NaN,1.0,12.0,Miso,2.0,-1.0,176.0
3,ENSG00000000003,ENST00000614008,-1,X,100636793,100637104,100636793.0,100637104.0,NaN,NaN,NaN,NaN,Miso,1.0,-2.0,311.0
4,ENSG00000000003,ENST00000614008,-1,X,100635558,100635746,100635570.0,100635746.0,NaN,NaN,1.0,12.0,Miso,2.0,-1.0,176.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82355,ENSG00000287542,ENST00000512194,1,4,88605795,88606049,88605795.0,88605823.0,NaN,NaN,1.0,226.0,Siso,4.0,-1.0,28.0
82356,ENSG00000288920,ENST00000484897,1,19,23763004,23763134,23763004.0,23763134.0,NaN,NaN,NaN,NaN,Siso,1.0,-4.0,130.0
82357,ENSG00000288920,ENST00000484897,1,19,23807826,23807939,23807826.0,23807939.0,NaN,NaN,NaN,NaN,Siso,2.0,-3.0,113.0
82358,ENSG00000288920,ENST00000484897,1,19,23808719,23808814,23808719.0,23808814.0,NaN,NaN,NaN,NaN,Siso,3.0,-2.0,95.0


In [31]:
def exon_ord(df):

    
    df['Exon_nb'] = list(range(1,df.shape[0]+1))

    return df
    

# Fct to compute intron boundaries
def get_introns(df):
    
#     if df.Strand.values[0] == 1:
    df = df.sort_values(by=['Exon region start (bp)', 'Exon region end (bp)'])

    l = list()
    exons = df['ranges'].values.tolist()
    for j, e in enumerate(exons):
        # Exon 1
        if j == 0:
            l.append(int(e.split("-")[1]) + 1)
            
        # Exon 2 <-> Exon -2
        elif j > 0 and j < len(exons) - 1:
            l.append(int(e.split("-")[0]) - 1)
            l.append(int(e.split("-")[1]) + 1)
            
        # Exon -1
        elif j == len(exons) - 1:
            l.append(int(e.split("-")[0]) - 1)

    # Final list        
    l = ["{}-{}".format(e, l[j + 1]) for j, e in enumerate(l) if j < len(l) - 1 if j % 2 == 0]
    df['Introns'] = l + [np.nan]

    
    if df.Strand.values[0] == -1:
        df = df.sort_values(by=['Exon region start (bp)', 'Exon region end (bp)'], ascending=False)

        df.loc[df['Introns'].isna() == False,  'Ordinal_nb'] = range(1,len(l)+1) 
        df['Ordinal_nb_inverted'] = np.array(list(range(1,df.shape[0] + 1))[::-1]) * -1
    
    if df.Strand.values[0] == 1:
        df = df.sort_values(by=['Exon region start (bp)', 'Exon region end (bp)'])

        df.loc[df['Introns'].isna() == False,  'Ordinal_nb'] = range(1,len(l)+1) 
        df['Ordinal_nb_inverted'] = (np.array(list(range(1,df.shape[0] + 1))[::-1]) * -1) + 1

#         df['Introns_nb'] = range(1,len(l)+2)    

    return df[['Miso_siso', 'GeneID', 'transcript_id', 'Strand', 'Chromosome/scaffold name', 'Introns', 'Ordinal_nb', 'Ordinal_nb_inverted']].dropna()
#     return df[['GeneID', 'transcript_id', 'Strand', 'Chromosome/scaffold name', 'Introns', 'Introns_nb']]


exon_tmp_dev = exon_miso_siso_to_keep[['GeneID', 'transcript_id', 'Strand', 'Chromosome/scaffold name', 'Exon region start (bp)', 'Exon region end (bp)', "5' UTR start", "5' UTR end", "3' UTR start", "3' UTR end", 'CDS start', 'CDS end', 'Miso_siso']]
exon_tmp_dev = exon_tmp_dev.sort_values(by=['Chromosome/scaffold name', 'Exon region start (bp)', 'Exon region end (bp)', 'GeneID', 'transcript_id'])
exon_tmp_dev['ranges'] = exon_tmp_dev['Exon region start (bp)'].astype(str) + '-' + exon_tmp_dev['Exon region end (bp)'].astype(str)

# exon_tmp_dev = exon_tmp_dev.loc[exon_tmp_dev['GeneID'].isin(exon_tmp_dev['GeneID'].unique().tolist()[:100])]



introns_df = exon_tmp_dev.groupby(['Miso_siso', 'GeneID', 'transcript_id']).progress_apply(get_introns).reset_index(drop=True)
introns_df['Chromosome/scaffold name'] = introns_df['Chromosome/scaffold name'].astype(str)
introns_df["Length"] = introns_df["Introns"].apply(lambda r: int(r.split("-")[1]) - int(r.split("-")[0]))


# introns_df.to_sql('Introns', engine, if_exists='replace')
introns_df.to_parquet('/gstock/GeneIso/V2/Introns.parquet')

introns_df



100%|██████████| 47544/47544 [17:01<00:00, 46.53it/s] 


,Miso_siso,GeneID,transcript_id,Strand,Chromosome/scaffold name,Introns,Ordinal_nb,Ordinal_nb_inverted,Length
0,Miso,ENSG00000000003,ENST00000373020,-1,X,100635747-100636607,1.0,-7,860
1,Miso,ENSG00000000003,ENST00000373020,-1,X,100635253-100635557,2.0,-6,304
2,Miso,ENSG00000000003,ENST00000373020,-1,X,100634030-100635177,3.0,-5,1147
3,Miso,ENSG00000000003,ENST00000373020,-1,X,100633540-100633930,4.0,-4,390
4,Miso,ENSG00000000003,ENST00000373020,-1,X,100632569-100633404,5.0,-3,835
...,...,...,...,...,...,...,...,...,...
488800,Siso,ENSG00000287542,ENST00000512194,1,4,88704282-88704507,24.0,-3,225
488801,Siso,ENSG00000287542,ENST00000512194,1,4,88704611-88706751,25.0,-2,2140
488802,Siso,ENSG00000288920,ENST00000484897,1,19,23763135-23807825,1.0,-4,44690
488803,Siso,ENSG00000288920,ENST00000484897,1,19,23807940-23808718,2.0,-3,778
